05a · Negative Mining

Purpose This notebook mines real background patches (non-nodules) from LUNA16 CT scans to create a harder, more realistic negative set. We enforce distance from physician-marked nodules and (when available) restrict to lung regions via masks or HU heuristics.

Key steps include:

- Mapping scans and optional lung masks; resampling to 1 mm isotropic
- Proposing background centers inside lung and ≥ 15 mm from any nodule
- Extracting 64³ HU-windowed cubes and writing them to disk
- Streaming an index (bg_index.csv) with file names, centers, and metadata

In this cell we are going to set up paths for the raw LUNA16 data and the working output, assert the dataset is present, create an output folder for background (non-nodule) 64³ patches, and load the physician annotations.csv to know where the true nodules are (for later exclusion and distance checks).

In [ ]:
import gc
import math 
import json
import random
import numpy as np
import pandas as pd
import SimpleITK as sitk

from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
RAW = Path("/kaggle/input/luna16")
assert RAW.exists()

WORK = Path("/kaggle/working")
OUT_DIR = WORK / "bg_patches_64mm"
OUT_DIR.mkdir(exist_ok=True)

ann = pd.read_csv(RAW / "annotations.csv")
print("annotations:", ann.shape)

annotations: (1186, 5)


In this cell we are going to discover all CT volumes by scanning for .mhd files (excluding segmentation/mask files), build a mapping from series UID → scan path, and optionally map any available lung masks (files containing “seg”/“mask”) to the same UID. We then print counts to verify how many scans and masks were found before mining negatives.

In [ ]:
# find all .mhd scans recursively; exclude masks for the CT map
all_mhd = list(RAW.rglob("*.mhd"))
scan_paths = {p.stem: p for p in all_mhd if "seg" not in p.name.lower()}

# try to find a lung mask per scan (optional, improves quality)
mask_paths = {}
for p in all_mhd:
    name = p.name.lower()
    if "seg" in name or "mask" in name:
        mask_paths[p.stem.replace("_seg","").replace("_mask","")] = p

len_scans = len(scan_paths)
len_masks = len(mask_paths)

print(f"CT scans mapped: {len_scans} | masks found: {len_masks}")

CT scans mapped: 888 | masks found: 0


In this cell we are going to define core utilities:

- resample_sitk to resample a CT to 1×1×1 mm spacing for consistent voxel geometry,
- hu_window to clip and normalize intensities to a lung-friendly HU window
- extract_cube to safely crop a 64×64×64 cube around a center (handling edge padding when the crop would go out of bounds). These helpers will be used to standardize scans and extract background patches.

In [ ]:
def resample_sitk(image, new_spacing=(1.0,1.0,1.0)):
    old_sp = np.array(image.GetSpacing()[::-1])
    old_sz = np.array(image.GetSize()[::-1])
    new_sz = (old_sp * old_sz / np.array(new_spacing)).round().astype(int)

    f = sitk.ResampleImageFilter()
    f.SetInterpolator(sitk.sitkLinear)
    f.SetOutputSpacing(tuple(new_spacing[::-1]))  
    f.SetSize([int(s) for s in new_sz[::-1]]
    f.SetOutputDirection(image.GetDirection())
    f.SetOutputOrigin(image.GetOrigin())
    return f.Execute(image)

def hu_window(arr, hu_min=-1000, hu_max=400):
    arr = np.clip(arr, hu_min, hu_max)
    return (arr - hu_min) / (hu_max - hu_min)

def extract_cube(vol, center, size=64):
    z,y,x = center
    r = size//2
    z1,z2 = z-r, z+r
    y1,y2 = y-r, y+r
    x1,x2 = x-r, x+r
    pad = [[0,0],[0,0],[0,0]]
    def clip(lo, hi, dim, pi):
        if lo < 0:
            pad[pi][0] = -lo; lo = 0
        if hi > dim:
            pad[pi][1] = hi - dim; hi = dim
        return lo, hi
    z1,z2 = clip(z1,z2, vol.shape[0], 0)
    y1,y2 = clip(y1,y2, vol.shape[1], 1)
    x1,x2 = clip(x1,x2, vol.shape[2], 2)
    cube = vol[z1:z2, y1:y2, x1:x2]
    if any(p for pair in pad for p in pair):
        cube = np.pad(cube, pad, mode="constant")
    return cube.astype(np.float32, copy=False)

In this cell we are going to organize positives and set up fast I/O for scans. We group annotations.csv by seriesuid, define a world_to_vox converter (world mm → voxel indices), and create a scan cache. The loader load_scan_and_centres(seriesuid) reads a CT once, resamples it to 1 mm isotropic, windows HU to [−1000, 400] → [0,1], optionally loads/resamples a lung mask, converts all positive nodule coordinates to voxel space, and caches {vol_norm, lung_mask, spacing_zyx, origin_zyx, pos_vox} for reuse.

In [ ]:
# group annotations by series
pos_by_series = {s: df.copy() for s, df in ann.groupby("seriesuid")}
series_uids = list(pos_by_series.keys())

def world_to_vox(coords_world, origin_zyx, spacing_zyx):
    return np.round((coords_world - origin_zyx) / spacing_zyx).astype(int)

# build a small cache so each scan is only read/resampled once
scan_cache = {}

def load_scan_and_centres(seriesuid):
    """Returns dict with vol_norm (float32 0-1), lung_mask (0/1 or None),
       spacing_zyx, origin_zyx, pos_vox: [N,3] voxel coords."""
    if seriesuid in scan_cache:
        return scan_cache[seriesuid]

    p = scan_paths.get(seriesuid)
    if p is None:
        return None

    img = sitk.ReadImage(str(p))
    iso = resample_sitk(img, (1,1,1))
    vol = sitk.GetArrayFromImage(iso).astype(np.int16)
    vol_norm = hu_window(vol)

    # origin/spacing in z,y,x order
    spacing_zyx = np.array(iso.GetSpacing()[::-1])
    origin_zyx  = np.array(iso.GetOrigin()[::-1])

    # load mask if available; resample to same grid & binarize
    lung_mask = None
    mp = mask_paths.get(seriesuid)
    if mp is not None:
        mimg = sitk.ReadImage(str(mp))
        m_iso= resample_sitk(mimg, (1,1,1))
        lung_mask = sitk.GetArrayFromImage(m_iso)
        # Some masks are {0,255} or {0,1}; binarize
     lung_mask = (lung_mask > np.percentile(lung_mask, 50)).astype(np.uint8)

    # convert all positive world coords to voxel coords on the iso grid
    sdf = pos_by_series[seriesuid]
    pos_world = sdf[["coordZ","coordY","coordX"]].to_numpy()
    pos_vox   = world_to_vox(pos_world, origin_zyx, spacing_zyx)

    out = dict(vol_norm=vol_norm, lung_mask=lung_mask,
               spacing_zyx=spacing_zyx, origin_zyx=origin_zyx,
               pos_vox=pos_vox)
    scan_cache[seriesuid] = out
    return out

print("Prepared to cache", len(series_uids), "scans")

Prepared to cache 601 scans


In this cell we are going to implement negative-center proposal logic. sample_bg_centers randomly proposes voxel centers that (a) lie inside lung if a mask exists (with a small local-mask occupancy check), (b) are at least min_dist_mm (default 15 mm) away from any positive nodule, and (c) are far enough from borders to extract a full 64³ cube. It returns up to n_samples viable background centers per scan, with a cap on attempts to avoid infinite loops.

In [5]:
def sample_bg_centers(scan, n_samples=2, min_dist_mm=15, size=64, max_tries=5000):
    """Return up to n_samples voxel centers inside lung and far from positives."""
    vol   = scan["vol_norm"]
    mask  = scan["lung_mask"]
    pos   = scan["pos_vox"]
    Z,Y,X = vol.shape
    r = size//2
    min_dist_vox = int(round(min_dist_mm / 1.0))  # 1mm voxels -> 1:1

    centers = []
    tries = 0
    while len(centers) < n_samples and tries < max_tries:
        tries += 1
        # keep within bounds so 64^3 can be cut without pad preference
        z = random.randint(r, Z-r-1)
        y = random.randint(r, Y-r-1)
        x = random.randint(r, X-r-1)
        c = np.array([z,y,x])

        # inside-lung constraint (if mask available)
        if mask is not None:
            if mask[z,y,x] == 0:
                continue
            # also require a minimum local lung presence
            if mask[max(0,z-3):z+4, max(0,y-3):y+4, max(0,x-3):x+4].sum() < 10:
                continue

        # far from all positives
        if pos.size > 0:
            d = np.linalg.norm(pos - c, axis=1).min()
            if d < min_dist_vox:
                continue

        centers.append(c)
    return centers

In this cell we are going to add streaming, scan-sliced patch extraction helpers to avoid loading entire volumes when writing many negatives. resample_to_iso resamples any SimpleITK image to 1×1×1 mm (linear by default). roi_patch pulls a 64×64×64 region of interest around a requested (z,y,x) center directly from the resampled image, pads if the crop hits borders, then applies a lung HU window [−1000, 400] and scales to [0,1] float16. This keeps mining memory-efficient and fast while producing uniformly shaped background cubes.

In [ ]:
OUT_DIR = Path("/kaggle/working/bg_patches_64mm"); OUT_DIR.mkdir(exist_ok=True)
out_csv = Path("/kaggle/working/bg_index.csv")

if out_csv.exists(): 
    out_csv.unlink()

def resample_to_iso(img, spacing=(1.0,1.0,1.0), linear=True):
    f = sitk.ResampleImageFilter()
    f.SetInterpolator(sitk.sitkLinear if linear else sitk.sitkNearestNeighbor)
    f.SetOutputSpacing(spacing)
    size = img.GetSize()
    sp   = img.GetSpacing()
    new_size = [int(round(size[i]*sp[i]/spacing[i])) for i in range(3)]
    f.SetSize(new_size)
    f.SetOutputDirection(img.GetDirection())
    f.SetOutputOrigin(img.GetOrigin())
    return f.Execute(img)

def roi_patch(img_iso, center_zyx, size=64):
    """Return 64³ float16 patch (HU-windowed to [-1000,400] -> [0,1]) without loading whole scan."""
    z,y,x = map(int, center_zyx)
    r = size//2
    # SimpleITK uses (x,y,z) for index & size
    start = [max(0, x-r), max(0, y-r), max(0, z-r)]
    stop  = [min(img_iso.GetSize()[0], x+r), min(img_iso.GetSize()[1], y+r), min(img_iso.GetSize()[2], z+r)]
    roi_sz= [stop[0]-start[0], stop[1]-start[1], stop[2]-start[2]]

    roi = sitk.RegionOfInterest(img_iso, size=roi_sz, index=start)
    arr = sitk.GetArrayFromImage(roi).astype(np.int16)     # (z,y,x)
    # pad back to 64³ if near borders
    pad = [(0,0),(0,0),(0,0)]
    for ax,need in enumerate([size-arr.shape[0], size-arr.shape[1], size-arr.shape[2]]):
        if need>0:
            before = need//2; after = need-before; pad[ax]=(before, after)
    if any(a or b for a,b in pad): arr = np.pad(arr, pad, mode="constant", constant_values=-1000)

    # HU window -> float16 [0,1]
    arr = np.clip(arr, -1000, 400)
    arr = ((arr + 1000) / 1400.0).astype(np.float16, copy=False)
    return arr  # (64,64,64) float16

In this cell we are going to perform the actual mining loop. For each seriesuid, we load and resample the CT to 1 mm, compute (spacing, origin) and convert positive nodule coordinates to voxel space. We optionally resample a lung mask. We then repeatedly sample candidate centers that (a) stay within bounds for a 64³ crop, (b) are inside lung (mask check or a small 7³ HU patch heuristic), and (c) are at least MIN_DIST_MM from all positives. Up to NEG_PER_SCAN valid centers are accepted. For each center we extract a 64×64×64 ROI (roi_patch), save it as {seriesuid}_bg{i}.npy, and append a row to bg_index.csv (streaming with a header only once). We also track counts, skip scans with no valid centers, and free memory (delete SITK objects and call gc.collect()) each iteration. Configuration knobs: NEG_PER_SCAN (start small, e.g., 1–4) and MIN_DIST_MM (e.g., 15 mm) control dataset size and hardness.

In [ ]:
def world_to_vox(world_zyx, origin_zyx, spacing_zyx):
    return np.round((world_zyx - origin_zyx) / spacing_zyx).astype(int)

NEG_PER_SCAN = 1     
MIN_DIST_MM  = 15

header_written = False
saved, skipped = 0, 0

series_uids = list(ann['seriesuid'].unique())

for suid in tqdm(series_uids, desc="mining", mininterval=1.0):
    ct_path = scan_paths.get(suid)
    if ct_path is None:
        skipped += 1; continue

    # load CT as SimpleITK & resample to 1mm
    img = sitk.ReadImage(str(ct_path))
    iso = resample_to_iso(img, (1.0,1.0,1.0), linear=True)

    # spacing/origin (x,y,z order from sitk) -> convert to (z,y,x)
    sp_xyz = np.array(iso.GetSpacing()); org_xyz = np.array(iso.GetOrigin())
    spacing_zyx = sp_xyz[::-1]; origin_zyx = org_xyz[::-1]

    # positives for distance check (in voxels) – numbers only
    sdf = ann[ann.seriesuid==suid]
    pos_world = sdf[['coordZ','coordY','coordX']].to_numpy()
    pos_vox   = world_to_vox(pos_world, origin_zyx, spacing_zyx) if len(pos_world) else np.empty((0,3), int)
    min_dist_vox = int(round(MIN_DIST_MM/1.0))  # 1mm vox -> 1:1

    # mask for inside-lung check (kept as SimpleITK, ROI on demand)
    mask_img = None
    mp = mask_paths.get(suid)
    if mp is not None:
        mask_img = resample_to_iso(sitk.ReadImage(str(mp)), (1.0,1.0,1.0), linear=False)

    Z,Y,X = iso.GetSize()[2], iso.GetSize()[1], iso.GetSize()[0]
    r = 32 

    def valid_center(c_zyx):
        z,y,x = c_zyx
        # bounds with margin for 64³
        if not (r <= z < Z-r and r <= y < Y-r and r <= x < X-r):
            return False
        # inside lung? (mask if available; else HU heuristic)
        if mask_img is not None:
            m = sitk.RegionOfInterest(mask_img, [1,1,1], [int(x),int(y),int(z)])
            if sitk.GetArrayFromImage(m)[0,0,0] == 0:
                return False
        else:
            # read tiny 3³ neighbourhood and require mean HU be lung-like
            tiny = roi_patch(iso, (z,y,x), size=7)      # float16 [0,1]
            if float(tiny.mean()) > 0.6:                # > ~ -160 HU -> likely not lung
                return False
        # far from positives
        if pos_vox.size:
            if np.min(np.linalg.norm(pos_vox - np.array([z,y,x]), axis=1)) < min_dist_vox:
                return False
        return True

    # sample a few valid centres
    centers, tries = [], 0
    while len(centers) < NEG_PER_SCAN and tries < 4000:
        tries += 1
        c = np.array([random.randint(r, Z-r-1),
                      random.randint(r, Y-r-1),
                      random.randint(r, X-r-1)])
        if valid_center(c):
            centers.append(c)

    if not centers:
        skipped += 1
        # free memory explicitly
        del img, iso
        if mask_img is not None: del mask_img
        gc.collect()
        continue

    # save patches + append to CSV (streaming)
    rows = []
    for i, c in enumerate(centers):
        cube = roi_patch(iso, c, size=64)                 # float16 (64³)
        fname = f"{suid}_bg{i}.npy"
        np.save(OUT_DIR / fname, cube)                    # ~0.5MB each
        rows.append({
            "patch_file": fname, "seriesuid": suid, "label": 0,
            "center_z": int(c[0]), "center_y": int(c[1]), "center_x": int(c[2]),
            "min_dist_mm": MIN_DIST_MM,
            "has_lung_mask": int(mask_img is not None)
        })
        saved += 1

    pd.DataFrame(rows).to_csv(out_csv, mode="a", index=False, header=not header_written)
    header_written = True

    # free per-scan objects
    del img, iso, rows, centers
    if mask_img is not None: del mask_img
    gc.collect()

print(f"✅ saved {saved} bg patches to {OUT_DIR} | skipped scans: {skipped}")

mining:   0%|          | 0/601 [00:00<?, ?it/s]

✅ saved 601 bg patches to /kaggle/working/bg_patches_64mm | skipped scans: 0
